In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
df = sqlContext.read.load('file:///home/cloudera/Downloads/big-data-4/daily_weather.csv',format='com.databricks.spark.csv',header='true',inferSchema='true')

In [ ]:
df.columns

In [ ]:
#Sort desc
from pyspark.sql.functions import desc
df.sort(desc('avg_wind_speed_9am')).select('avg_wind_speed_9am').show(1)

In [ ]:
#Max 
df.groupby().max('avg_wind_speed_9am').show()

In [ ]:
#Get count of null values of single column in pyspark
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(col('rain_accumulation_9am').isNull(),True))]).show()

In [ ]:
#Get correlation between two cols
df.stat.corr('relative_humidity_9am', 'relative_humidity_3pm')

In [ ]:
df.select('air_temp_9am').show()

In [ ]:
#Get a max and then get other stuff, agg just gets 1 column
data = df.agg({'max_wind_speed_9am':'max'}).alias('max_wind_speed_9am')
data = data.select(col('max(max_wind_speed_9am)').alias('max_wind_speed_9am'))
data_joined = df.join(data, 'max_wind_speed_9am').select('max_wind_speed_9am','max_wind_direction_9am')
data_joined.show()

In [ ]:
#Drop missing values
removedNADf = df.na.drop()

In [ ]:
#If we remove all missing values from the data, how many air pressure at 9am measurements have values between 911.736 and 914.67?
removedNADf.filter('air_pressure_9am > 911.736 and air_pressure_9am < 914.67').count()

In [ ]:
#If we impute the missing values with the minimum value, how many air temperature at 9am measurements are less than 42.292?
df.select('air_temp_9am').na.fill(df.agg({'air_temp_9am':'min'}).collect()[0][0]) \ 
.filter('air_temp_9am < 42.292').count()

In [ ]:
#How many samples have missing values for air_pressure_9am?
df.select([count(when(isnan('air_pressure_9am') | col('air_pressure_9am').isNull(),True))]).show()

In [ ]:
#Which column in the weather dataset has the most number of missing values?
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()